In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import pandas as pd
import random
import wikipedia as wp
from wikipedia.exceptions import DisambiguationError, PageError
import networkx as nx
import matplotlib.pyplot as plt
from langsmith import Client
from langchain_core.tracers.context import tracing_v2_enabled
import os

# Now you can access the variables using os.environ
api_key = os.environ.get("LANGCHAIN_API_KEY")
tracing_v2 = os.environ.get("LANGCHAIN_TRACING_V2")
project_name = os.environ.get("LANGCHAIN_PROJECT")

# Use the variables in your script
os.environ["LANGCHAIN_API_KEY"] = api_key
os.environ["LANGCHAIN_TRACING_V2"] = tracing_v2
os.environ["LANGCHAIN_PROJECT"] = project_name

client = Client()


In [6]:
print(wp.summary("data science"))

Data science is an interdisciplinary academic field that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from potentially noisy, structured, or unstructured data.Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine). Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data. It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge. However, data science is different from computer science and information science. Turing Award winner Jim Gray imagined data s